In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
import os
from webdav4.fsspec import WebdavFileSystem
from pathlib import Path
import yaml

## Connect to Sciebo

In [4]:
load_dotenv()
USR = os.getenv('REPORT_USR')
PWD = os.getenv('REPORT_PWD')

## Accessing reports of consultants

In [5]:
fs = WebdavFileSystem("https://uni-bonn.sciebo.de/public.php/webdav", auth=(USR, PWD))
reports = fs.ls('/', detail=False)
assert len(reports) >0, "No reports found!"
consultants = [Path(report).stem for report in reports]
print(f'{len(reports)} reports found and the names of consultants are: {consultants}')

3 reports found and the names of consultants are: ['mohammad', 'nick', 'sangeetha']


## Creating data directory if it does not exist

In [6]:
os.makedirs('data/', exist_ok=True)

## Reading from data/consultants.txt if it exists

In [8]:
f_consultants = 'data/consultants.yaml'
if Path(f_consultants).exists():
    with open(f_consultants, 'r') as f:
        consultant_file = yaml.safe_load(f)
else:
    consultant_file = {}
    for r,c in zip(reports, consultants):
        consultant_file[c] = {'hash':'', 'download': 1}
consultant_file

{'mohammad': {'hash': '', 'download': 1},
 'nick': {'hash': '', 'download': 1},
 'sangeetha': {'hash': '', 'download': 1}}

## Updating data/consultants.yaml if remote file has changed

In [9]:
ukey = lambda r: fs.ukey(r)
for r, (k, v) in zip(reports, consultant_file.items()):
    if ukey(r) != v['hash']:
        print(f"{r} has changed on remote. Updating hash")
        consultant_file[k]['hash'] = ukey(r)
        consultant_file[k]['download'] = 1
        continue
    print(f"{r} has not changed on remote.")
    consultant_file[k]['download'] = 0

with open(f_consultants, 'w') as f:
    yaml.dump(consultant_file, f, default_flow_style=False)

mohammad.docx has changed on remote. Updating hash
nick.docx has changed on remote. Updating hash
sangeetha.docx has changed on remote. Updating hash
